In [22]:
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [111]:
train_image_dir = '재활용품_분류_및_선별_데이터/01-1.정식개방데이터/Validation/01.원천데이터/VS_1.영상추출_01.금속캔_001.철캔'
train_label_dir = '재활용품_분류_및_선별_데이터/01-1.정식개방데이터/Validation/02.라벨링데이터/VL_1.영상추출_01.금속캔_001.철캔'
img_height, img_width = 150, 150  # 원하는 이미지 크기

In [112]:
images = []
image_labels = []

In [113]:
# 디렉토리 내의 JSON 파일 탐색
for json_file in os.listdir(train_label_dir):
    if json_file.endswith('.json'):
        json_path = os.path.join(train_label_dir, json_file)
        
        # JSON 파일에서 라벨 데이터 읽기
        with open(json_path, encoding='utf-8') as f:
            data = json.load(f)
        
        img_file = data['IMAGE_INFO']['FILE_NAME']
        img_path = os.path.join(train_image_dir, img_file)
        
        # 이미지 파일이 실제로 존재하는지 확인
        if os.path.exists(img_path) and img_file.endswith('.jpg'):
            try:
                img = Image.open(img_path)  # 이미지를 실제로 불러와 확인
                annotations = data.get('ANNOTATION_INFO', [])
                for annotation in annotations:
                    points = annotation.get('POINTS', [None])[0]  # POINTS 리스트의 첫 번째 항목을 사용
                    if points and all(isinstance(x, (int, float)) for x in points):
                        images.append(img_path)  # 중복 허용: 이미지 경로를 여러 번 추가
                        image_labels.append((img_path, points, annotation))   # 이미지 경로와 포인트를 함께 저장
                # 라벨 수 출력
                print(f"File: {json_file}, Labels: {len(annotations)}")
            except (IOError, SyntaxError) as e:
                print(f"Error loading image {img_path}: {e}")

File: 1010599@0_01001_220906_P1_T3__0166.json, Labels: 1
File: 1010653@0_01001_220906_P1_T3__0163.json, Labels: 1
File: 1010670@0_01001_220906_P1_T3__1276.json, Labels: 1
File: 1010675@0_01001_220906_P1_T3__1281.json, Labels: 6
File: 1011559@0_01001_220906_P1_T3__2485.json, Labels: 1
File: 1011566@0_01001_220906_P1_T3__2465.json, Labels: 1
File: 1011583@0_01001_220906_P1_T3__0034.json, Labels: 1
File: 1012828@0_01001_220906_P1_T3__3499.json, Labels: 1
File: 1012959@0_01001_220906_P1_T3__1167.json, Labels: 1
File: 1012960@0_01001_220906_P1_T3__1171.json, Labels: 1
File: 1012964@0_01001_220906_P1_T3__1032.json, Labels: 1
File: 1012976@0_01001_220906_P1_T3__2605.json, Labels: 1
File: 1012978@0_01001_220906_P1_T3__2419.json, Labels: 1
File: 1012980@0_01001_220906_P1_T3__2609.json, Labels: 1
File: 1012982@0_01001_220906_P1_T3__2418.json, Labels: 2
File: 1013003@0_01001_220906_P1_T3__2606.json, Labels: 1
File: 1014335@0_01001_221006_P1_T3__0871.json, Labels: 1
File: 1014362@0_01001_221006_P1

In [114]:
print(f"Number of images: {len(images)}")
print(f"Number of labels: {len(image_labels)}")

Number of images: 3969
Number of labels: 3969


In [115]:
original_size = (1920, 1080)
target_size = (img_height, img_width)

In [116]:
def scale_points(points, original_size, target_size):
    # 원본 크기와 목표 크기 비율 계산
    scale_x = target_size[0] / original_size[0]
    scale_y = target_size[1] / original_size[1]

    # 포인트 스케일링
    scaled_points = [points[0] * scale_x, points[1] * scale_y, points[2] * scale_x, points[3] * scale_y]
    return scaled_points

In [117]:
def crop_image(img_path, points):
    img = Image.open(img_path)
    left, top, width, height = points
    right = left + width
    bottom = top + height

    # 포인트 유효성 검사
    img_width, img_height = img.size
    if left < 0 or top < 0 or right > img_width or bottom > img_height:
        raise ValueError(f"Invalid points for cropping: {points}")

    cropped_img = img.crop((left, top, right, bottom))
    return cropped_img

In [118]:
print(images[:1])

['재활용품_분류_및_선별_데이터/01-1.정식개방데이터/Validation/01.원천데이터/VS_1.영상추출_01.금속캔_001.철캔\\1010599@0_01001_220906_P1_T3__0166.jpg']


In [119]:
# image_labels 리스트의 구조 확인
print(image_labels[:1])  # 리스트의 처음 5개 항목 출력

[('재활용품_분류_및_선별_데이터/01-1.정식개방데이터/Validation/01.원천데이터/VS_1.영상추출_01.금속캔_001.철캔\\1010599@0_01001_220906_P1_T3__0166.jpg', [530.5541674833809, 571.54503755009, 173.75160238568606, 177.65076739562642], {'ID': '1', 'CLASS': '금속캔', 'DETAILS': '철캔', 'DAMAGE': '원형', 'DIRTINESS': '이물질(전체)', 'COVER': '없음', 'TRANSPARENCY': '불투명', 'SHAPE': '낮은 원통형', 'SHAPE_TYPE': 'BOX', 'POINTS': [[530.5541674833809, 571.54503755009, 173.75160238568606, 177.65076739562642]]})]


In [127]:
train_images = []
train_labels = []
failed_images = []

In [128]:
# 이미지 크롭 및 배열로 변환
for img_path, points, annotation in image_labels:
    try:
        # 포인트 데이터의 길이 확인
        if len(points) != 4:
            raise ValueError(f"Points data does not contain 4 elements: {points}")

        scaled_points = scale_points(points, original_size, target_size)
        
        # 포인트 유효성 검사
        if any(p < 0 for p in scaled_points):
            raise ValueError(f"Scaled points have negative values: {scaled_points}")
        if scaled_points[0] + scaled_points[2] > original_size[0] or scaled_points[1] + scaled_points[3] > original_size[1]:
            raise ValueError(f"Scaled points exceed image boundaries: {scaled_points}")

        cropped_img = crop_image(img_path, scaled_points)
        cropped_img = cropped_img.resize(target_size)
        img_array = img_to_array(cropped_img)
        img_array = img_array / 255.0  # 정규화
        train_images.append(img_array)
        train_labels.append(annotation['CLASS'])  # 라벨 추가
    except Exception as e:
        print(f"Failed to process image {img_path} with error: {e}")
        # 실패한 이미지도 원본 상태로 추가
        try:
            img = Image.open(img_path)
            img = img.resize(target_size)
            img_array = img_to_array(img)
            img_array = img_array / 255.0  # 정규화
            train_images.append(img_array)
            train_labels.append(annotation['CLASS'])
        except Exception as inner_e:
            print(f"Failed to load original image {img_path} with error: {inner_e}")
            failed_images.append((img_path, points, str(inner_e)))


Failed to process image 재활용품_분류_및_선별_데이터/01-1.정식개방데이터/Validation/01.원천데이터/VS_1.영상추출_01.금속캔_001.철캔\1028536@0_01001_221006_P1_T3__2798.jpg with error: Points data does not contain 4 elements: [1326.349367088608, 1075.1354360846026]
Failed to process image 재활용품_분류_및_선별_데이터/01-1.정식개방데이터/Validation/01.원천데이터/VS_1.영상추출_01.금속캔_001.철캔\1086992@0_01001_221006_P1_T3__2701.jpg with error: Points data does not contain 4 elements: [146.77466527156082, 355.5401015895218]
Failed to process image 재활용품_분류_및_선별_데이터/01-1.정식개방데이터/Validation/01.원천데이터/VS_1.영상추출_01.금속캔_001.철캔\1087012@0_01001_221006_P1_T3__1352.jpg with error: Points data does not contain 4 elements: [353.2103501042272, 359.6534712873994]
Failed to process image 재활용품_분류_및_선별_데이터/01-1.정식개방데이터/Validation/01.원천데이터/VS_1.영상추출_01.금속캔_001.철캔\1087029@0_01001_221006_P1_T3__1126.jpg with error: Points data does not contain 4 elements: [1102.1451104100947, 975.4495268138801]
Failed to process image 재활용품_분류_및_선별_데이터/01-1.정식개방데이터/Validation/01.원천데이터/VS_1.영상

In [129]:
print(len(train_images))
print(len(failed_images))
print(len(train_labels))

3969
0
3969


In [130]:
print(len(image_labels), image_labels[0])

3969 ('재활용품_분류_및_선별_데이터/01-1.정식개방데이터/Validation/01.원천데이터/VS_1.영상추출_01.금속캔_001.철캔\\1010599@0_01001_220906_P1_T3__0166.jpg', [530.5541674833809, 571.54503755009, 173.75160238568606, 177.65076739562642], {'ID': '1', 'CLASS': '금속캔', 'DETAILS': '철캔', 'DAMAGE': '원형', 'DIRTINESS': '이물질(전체)', 'COVER': '없음', 'TRANSPARENCY': '불투명', 'SHAPE': '낮은 원통형', 'SHAPE_TYPE': 'BOX', 'POINTS': [[530.5541674833809, 571.54503755009, 173.75160238568606, 177.65076739562642]]})


In [131]:
# 각 라벨을 원-핫 인코딩하고 하나의 벡터로 결합하는 함수
def encode_and_combine_labels(label_lists):
    encoded_labels = []
    for labels in label_lists:
        unique_labels = set(labels)
        label_to_int = {label: i for i, label in enumerate(unique_labels)}
        integer_labels = [label_to_int[label] for label in labels]
        encoded_label = to_categorical(integer_labels, num_classes=len(unique_labels))
        encoded_labels.append(encoded_label)
    return np.hstack(encoded_labels)
    
# 모든 라벨 데이터를 리스트로 준비
label_lists = [
    [label_info[2]['CLASS'] for label_info in image_labels],  # label_info[2]는 라벨 딕셔너리
    # [label_info[2]['DETAILS'] for label_info in image_labels],
    # [label_info[2]['DAMAGE'] for label_info in image_labels],
    # [label_info[2]['DIRTINESS'] for label_info in image_labels],
    # [label_info[2]['COVER'] for label_info in image_labels],
    # [label_info[2]['TRANSPARENCY'] for label_info in image_labels],
    # [label_info[2]['SHAPE'] for label_info in image_labels],
    # [label_info[2]['SHAPE_TYPE'] for label_info in image_labels]
]

# 라벨 인코딩 및 결합
all_labels = encode_and_combine_labels(label_lists)

In [132]:
print(len(train_images))
print(len(all_labels))

3969
3969


In [133]:
# 데이터셋 분할
train_images, val_images, train_labels, val_labels = train_test_split(train_images, all_labels, test_size=0.2, shuffle=True)

train_labels = np.argmax(train_labels, axis=1)
val_labels = np.argmax(val_labels, axis=1)

In [136]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense


# 모델 정의 및 컴파일
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')  # 클래스 수에 따라 변경
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # 다중 클래스 분류 손실 함수
              metrics=['accuracy'])

In [139]:
import numpy as np

# 리스트 형태의 데이터를 numpy 배열로 변환
train_images = np.array(train_images)
train_labels = np.array(train_labels)
val_images = np.array(val_images)
val_labels = np.array(val_labels)

# 데이터 크기 확인
print(train_images.shape, train_labels.shape)
print(val_images.shape, val_labels.shape)


(3175, 150, 150, 3) (3175,)
(794, 150, 150, 3) (794,)


In [140]:
history = model.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels))

Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 41s 318ms/step - accuracy: 0.9485 - loss: 0.1458 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 23s 228ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 23s 233ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 27s 275ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 25s 252ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 23s 234ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 22s 224ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
100/100 ━━━━━━━━━━━━

In [141]:
model.save('my_model.keras')